## **Project: AI-Powered Job Insights Using RAG & FAISS**

### **📌 Introduction**
Finding the right job can be overwhelming. Job seekers and recruiters need **structured insights** into job trends, skills in demand, and salary distributions across locations. Traditional search mechanisms rely on keyword matching, which often leads to **irrelevant results**. This project aims to enhance **job insights** using **Retrieval-Augmented Generation (RAG)** and **FAISS-based indexing** to build a **powerful job search and analysis tool**.

---

### **🎯 Objective**
The goal of this project is to:
1. **Enhance job search** using **semantic search with FAISS indexing** instead of simple keyword-based searches.
2. **Use RAG (Retrieval-Augmented Generation)** with ChatGPT to provide **smart job recommendations** based on queries.
3. **Leverage Pandas AI Agent** to create **interactive visualizations**, like:
   - Most in-demand skills by city.
   - Salary comparison across locations and job roles.
   - Heatmaps of job availability.
   - Insights into recruiter trends.

---

## **🔍 What is RAG (Retrieval-Augmented Generation)?**
**RAG** is an advanced AI technique that **combines retrieval-based search with generative models** like ChatGPT. Instead of relying on static model knowledge, RAG allows the model to **fetch real-time, relevant information** from an external database (FAISS in our case) before generating a response.

**Key Benefits of RAG:**
✅ Retrieves **up-to-date** information.  
✅ Provides **contextually relevant** answers.  
✅ Reduces **hallucination** in generative AI.  

---

## **⚡ What is FAISS & Why FAISS for Job Data?**
**FAISS (Facebook AI Similarity Search)** is an efficient **vector search** library for **fast similarity searches**. 

🔹 Traditional databases are slow for **large-scale** text search.  
🔹 FAISS **embeds** job descriptions as **vectors** and allows **fast retrieval** based on meaning (not keywords).  
🔹 Ideal for **semantic search** of jobs based on user queries.

---

## **🚀 Features to Implement**
### **1️⃣ Semantic Search for Jobs**
- Convert job descriptions into **embeddings** using OpenAI’s embedding model.
- Store embeddings in **FAISS** for fast retrieval.
- Implement **semantic search** for job queries.

### **2️⃣ ChatGPT-based RAG**
- Use **retrieved job postings** from FAISS to **generate smart responses**.
- Enable users to ask **detailed queries**, such as:
  - _"What are the best-paying jobs for Data Engineers in Bangalore?"_
  - _"Which job roles require Azure & Python in Chennai?"_

### **3️⃣ AI-Powered Job Analytics**
- **Pandas AI Agent** for exploratory data analysis.
- **Top skills by city** (Bar chart).
- **Salary comparison by job role & location** (Heatmap).
- **Job availability across cities** (Geo-based visualization).

---

In [11]:
###. Install & Import Required Libraries
# Install necessary libraries (only run once)
!pip install pandas faiss-cpu sentence-transformers transformers openai



Defaulting to user installation because normal site-packages is not writeable


### **📌 Load & Prepare Job Dataset**


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
# Load the dataset
df = pd.read_csv("hirist.csv")

# Drop unnecessary columns
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# Function to clean HTML tags from text
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text() if pd.notna(text) else ""

# Apply the cleaning function to the Job Description column
df['Job Description'] = df['Job Description'].apply(clean_html)

# Display sample data
df.head()


C:\Users\Anuvadini-019\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Job ID,Job Description,Title,Designation,Min Experience,Max Experience,Category ID,Job URL,Location,Company ID,Company Name,Recruiter ID,Recruiter Name,Recruiter Designation,Min Salary,Max Salary,Functional Area,Skills
0,1432749,Job Description :Were Haleon. A new world-lead...,Haleon - Data Engineer - Azure Databricks/Data...,Data Engineer,5,9,7,https://www.hirist.com/j/haleon-data-engineer-...,Bangalore,3348,Hirist,180511,Talent Bridge,HR,15,27,Data Analysis / Business Analysis,"Data Engineering, Data Pipeline, Data Architec..."
1,1434028,Job Summary :We are looking for a fresher Data...,Data Analyst,Data Analyst,0,3,7,https://www.hirist.com/j/data-analyst-1434028....,Chennai,2767,VIBRANTUM LABZ PRIVATE LIMITED,194867,swetha,hr,6,12,Data Analysis / Business Analysis,"Data Analyst, Data Analytics, SQL, Reporting T..."
2,1433670,Job Title : Data AnalystExperience : 6 months ...,Data Analyst,Data Analyst,0,2,7,https://www.hirist.com/j/data-analyst-1433670....,Gurgaon/Gurugram,0,Creencia Technologies Pvt Ltd,57068,Aruna,Talent Acquisition Specialist,7,8,Data Analysis / Business Analysis,"Data Analyst, Data Analytics, Data Management,..."
3,1434327,Job Description :Our Client works with a varie...,Data Engineer - Python/Spark,Data Engineer,2,10,7,https://www.hirist.com/j/data-engineer-pythons...,Bangalore,0,Fidius Advisory,132726,Priyanka Mohan,Consultant,8,30,Data Engineering,"Data Engineering, Python, Spark, Scala, Data M..."
4,1433114,Job Description :- We are seeking a highly ski...,Data Engineer - Machine Learning,Data Engineer,7,10,7,https://www.hirist.com/j/data-engineer-machine...,Bangalore,0,Interon IT Solutions,212071,Charan Reddy,US IT Recruiter,20,30,Data Engineering,"Machine Learning, Data Engineering, ETL, Azure..."


In [3]:
# ============================
# 🔹 LOAD & PREPARE JOB DATASET
# ============================

"""
In this section, we prepare job listings in a structured format.
Each job has multiple fields (Title, Company, Location, Skills, Experience).
To make it searchable, we merge all fields into a single text string.
"""

# Convert job postings into a structured search-friendly format
df["combined_text"] = df.apply(lambda row: " | ".join([f"{col}: {row[col]}" for col in df.columns if col != "combined_text"]), axis=1)

# Display sample data
df.head()


,Job ID,Job Description,Title,Designation,Min Experience,Max Experience,Category ID,Job URL,Location,Company ID,Company Name,Recruiter ID,Recruiter Name,Recruiter Designation,Min Salary,Max Salary,Functional Area,Skills,combined_text
0,1432749,Job Description :Were Haleon. A new world-lead...,Haleon - Data Engineer - Azure Databricks/Data...,Data Engineer,5,9,7,https://www.hirist.com/j/haleon-data-engineer-...,Bangalore,3348,Hirist,180511,Talent Bridge,HR,15,27,Data Analysis / Business Analysis,"Data Engineering, Data Pipeline, Data Architec...",Job ID: 1432749 | Job Description: Job Descrip...
1,1434028,Job Summary :We are looking for a fresher Data...,Data Analyst,Data Analyst,0,3,7,https://www.hirist.com/j/data-analyst-1434028....,Chennai,2767,VIBRANTUM LABZ PRIVATE LIMITED,194867,swetha,hr,6,12,Data Analysis / Business Analysis,"Data Analyst, Data Analytics, SQL, Reporting T...",Job ID: 1434028 | Job Description: Job Summary...
2,1433670,Job Title : Data AnalystExperience : 6 months ...,Data Analyst,Data Analyst,0,2,7,https://www.hirist.com/j/data-analyst-1433670....,Gurgaon/Gurugram,0,Creencia Technologies Pvt Ltd,57068,Aruna,Talent Acquisition Specialist,7,8,Data Analysis / Business Analysis,"Data Analyst, Data Analytics, Data Management,...",Job ID: 1433670 | Job Description: Job Title :...
3,1434327,Job Description :Our Client works with a varie...,Data Engineer - Python/Spark,Data Engineer,2,10,7,https://www.hirist.com/j/data-engineer-pythons...,Bangalore,0,Fidius Advisory,132726,Priyanka Mohan,Consultant,8,30,Data Engineering,"Data Engineering, Python, Spark, Scala, Data M...",Job ID: 1434327 | Job Description: Job Descrip...
4,1433114,Job Description :- We are seeking a highly ski...,Data Engineer - Machine Learning,Data Engineer,7,10,7,https://www.hirist.com/j/data-engineer-machine...,Bangalore,0,Interon IT Solutions,212071,Charan Reddy,US IT Recruiter,20,30,Data Engineering,"Machine Learning, Data Engineering, ETL, Azure...",Job ID: 1433114 | Job Description: Job Descrip...


### **📌 Generate Embeddings for Jobs**


In [4]:
"""
To enable similarity search, we convert job descriptions into dense embeddings using
the `all-MiniLM-L6-v2` model from Sentence Transformers.
"""

# Load a pre-trained Sentence Transformer model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for all job postings
job_descriptions = df["combined_text"].tolist()
job_embeddings = embedding_model.encode(job_descriptions, convert_to_numpy=True)

# Check embedding shape (should be [num_jobs, embedding_dim])
print(f"Embedding Shape: {job_embeddings.shape}")


C:\Users\Anuvadini-019\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anuvadini-019\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Embedding Shape: (363, 384)


### **📌 Set Up FAISS Index for Fast Similarity Search**


In [5]:
"""
We use FAISS (Facebook AI Similarity Search) to create an index for efficient retrieval.
It allows fast approximate nearest neighbor (ANN) search.
"""

import faiss

# Define FAISS index
dimension = job_embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance search
index.add(job_embeddings)  # Add job embeddings to FAISS index

# Store metadata for each job (Job ID, Title, URL, Salary, etc.)
job_metadata_map = {
    i: {
        "Job ID": df.loc[i, "Job ID"],
        "Title": df.loc[i, "Title"],
        "Job URL": df.loc[i, "Job URL"],
        "Salary": f"{df.loc[i, 'Min Salary']} - {df.loc[i, 'Max Salary']} LPA",
        "Location": df.loc[i, "Location"],
        "Skills": df.loc[i, "Skills"],
        "Combined_text": df.loc[i, "combined_text"]
    }
    for i in range(len(df))
}

# Confirm index size
print(f"✅ FAISS index contains {index.ntotal} job entries.")


✅ FAISS index contains 363 job entries.


### **📌 Save FAISS Index Along With Meta Data**

In [6]:
import pickle
## Save the FAISS Index
#FAISS provides a built-in method to save the index as a binary file:

faiss.write_index(index, "faiss_index.bin")

## Save Job Metadata Separately
#Since FAISS only stores embeddings, we must **save job metadata separately** using Python's `pickle`

with open("job_metadata.pkl", "wb") as f:
    pickle.dump(job_metadata_map, f)

print("✅ FAISS index and metadata saved successfully!")


✅ FAISS index and metadata saved successfully!


### **📌 Implement Semantic Search**


In [7]:
# ============================
# 🔹 SEMANTIC SEARCH FUNCTION
# ============================

"""
This function takes a user's query, generates an embedding, and finds the closest jobs using FAISS.
It returns the top `k` matching jobs.
"""
# Function to generate embeddings
def get_embedding(text):
    return np.array(embedding_model.encode(text, normalize_embeddings=True))


# Function to search jobs using FAISS
def search_jobs(query, top_k=5):
    query_embedding = get_embedding(query)  # Convert query into an embedding
    distances, indices = index.search(query_embedding.reshape(1, -1), top_k)  # Search FAISS

    results = []
    for i, idx in enumerate(indices[0]):
        if idx == -1:
            continue  # Ignore invalid indices

        metadata = job_metadata_map[idx]  # Fetch job metadata
        combined_text = metadata["Combined_text"]
        #metadata.pop("Combined_text")
        score = round(1 / (1 + distances[0][i]), 4)  # Convert L2 distance to similarity score

        results.append({"metadata":metadata, "score":score, "combined_text":combined_text})

    return results

# Example Query
query = "Looking for a remote AI job with deep learning expertise"
results = search_jobs(query)

# Print results
print("\nTop Matching Jobs:")
for row in results:
    print(f"Match Score: {row['score']}\n{row['metadata']}\n{'-'*50}")



Top Matching Jobs:
Match Score: 0.5789999961853027
{'Job ID': np.int64(1429997), 'Title': 'Associate Engineering Manager/Engineering Manager - Artificial Intelligence/Machine Learning ', 'Job URL': 'https://www.hirist.com/j/associate-engineering-managerengineering-manager-artificial-intelligencemachine-learning-1429997.html', 'Salary': '40 - 50 LPA', 'Location': 'Gurgaon/Gurugram', 'Skills': 'Machine Learning, Deep Learning, Computer Vision, Neural Networks, Artificial Intelligence, Python, SQL', 'Combined_text': 'Job ID: 1429997 | Job Description: Role Overview :We are looking for an Associate Engineering Manager / Engineering Manager - AI/ML with 7-10 years of hands on experience in Artificial Intelligence (AI), Machine Learning (ML), Deep Learning, Computer Vision, and Neural Networks. The ideal candidate should have strong expertise in developing AI/ML models from scratch, along with coding and deployment experience. This is a leadership role that requires technical excellence, ha

### **📌 Test Queries**


In [8]:
test_queries = [
    "AI engineer role with experience in reinforcement learning and NLP",
    "Remote job for a Python developer with cloud computing expertise",
    "Senior data scientist position in Bangalore with at least 5 years of experience",
    "Startup job with focus on LLM, generative AI, and multi-modal learning",
    "Internship in machine learning with mentorship from experienced researchers"
]

for i, q in enumerate(test_queries):
    print(f"\n🔹 Test Query {i+1}: {q}")
    results = search_jobs(q)  # Get results

    for row in results:  # Loop through results (dictionaries)
        job = row["metadata"]  # Extract metadata
        score = row["score"]  # Extract match score

        print(f"  - Match Score: {score}")
        print(f"  - Title: {job['Title']}")
        print(f"  - Job URL: {job['Job URL']}")
        print(f"  - Location: {job['Location']}")
        print(f"  - Skills: {job['Skills']}")
        print("-" * 50)



🔹 Test Query 1: AI engineer role with experience in reinforcement learning and NLP
  - Match Score: 0.5559999942779541
  - Title: Generative AI Developer - NLP/Neural Networks 
  - Job URL: https://www.hirist.com/j/generative-ai-developer-nlpneural-networks-1401993.html
  - Location: Bangalore
  - Skills: Generative AI, NLP, Neural Networks, Deep Learning, Artificial Intelligence, Machine Learning, Python, Tensorflow, PyTorch, Version Control System, MLOps
--------------------------------------------------
  - Match Score: 0.5468999743461609
  - Title: Senior Data Scientist - Generative AI 
  - Job URL: https://www.hirist.com/j/senior-data-scientist-generative-ai-1427666.html
  - Location: Bangalore
  - Skills: Data Scientist, Data Science, Generative AI, NLP, Machine Learning, Artificial Intelligence, Tensorflow, LLM, Spark, Hadoop, Data Visualization
--------------------------------------------------
  - Match Score: 0.5454999804496765
  - Title: Accenture - Consultant - LLM/Generat

In [ ]:
### Integrate ChatGPT for RAG
import openai

openai.api_key = "PUT_YOUR_OPENAI_API_KEY"

def generate_rag_response(query):
    retrieved_jobs = search_jobs(query, 10)
    job_context = "\n".join([row["combined_text"] for row in retrieved_jobs])
    
    prompt = f"""
    You are an AI job assistant. You will be asked a question related to jobs, provide your response based over the context provided only.
    Do not mention anything about context.
    Here is the context for the give job query:
    {job_context}
    
    Answer the user's query: {query}
    """
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content.strip()

# Example RAG-based question
print(generate_rag_response("What are the top-paying Data Engineering jobs in Bangalore?"))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Example RAG-based question
print(generate_rag_response("Tell me top sought skills for a fresh graduate related to data analyst role that I should learn"))


For a fresh graduate aiming for a data analyst role, the top sought skills to learn include:

1. **Excel Proficiency**: Understanding data formatting, sorting, and using functions like pivot tables and IF statements.
2. **SQL**: Familiarity with SQL commands for data retrieval and manipulation, as it's essential for working with databases.
3. **Data Visualization**: Experience with tools like Tableau or Power BI for creating interactive dashboards and reports.
4. **Analytical Thinking**: Ability to interpret data patterns and draw meaningful conclusions from datasets.
5. **Programming Skills**: Basics of Python or R for data manipulation and analysis.
6. **Statistical Knowledge**: Understanding basic statistical concepts and methodologies for data analysis.
7. **Data Cleaning and Preparation**: Skills in filtering, cleaning, and organizing data to ensure accuracy and quality.
8. **Communication Skills**: Ability to clearly explain findings and data insights to non-technical stakeholder

In [ ]:
# Example RAG-based question
print(generate_rag_response("""I did a course on powerbi, suggest me what shall I learn more 
                                and give me some jobs(along with job link) where i can apply after learning those extra skills.
                                Also create skill section that I can add to my resume for each of the job to have better chances of getting shortlisted."""))

To enhance your Power BI skills and improve your job prospects, consider focusing on the following areas:

1. **DAX (Data Analysis Expressions)**: Learn to write complex formulas for data calculations and analysis in Power BI.
2. **Data Modeling**: Understand how to create and optimize data models to support business intelligence needs.
3. **SQL Proficiency**: Gain strong skills in SQL for data extraction and manipulation.
4. **ETL (Extract, Transform, Load)**: Familiarize yourself with ETL processes to handle data more effectively.
5. **Data Visualization Best Practices**: Master the principles of designing effective visualizations.
6. **Azure Data Services**: Knowledge of Azure services related to data storage and analytics can be beneficial.
7. **Power Query**: Learn to clean and transform data using Power Query.

Here are some job opportunities that align with enhancing your skills:

1. **Data Analyst - Power BI**
   - Job Link: [Data Analyst - Power BI](https://www.hirist.com/j/da

---

## **📌 Summary & Conclusion for Notebook**
These sections summarize your analysis and provide key takeaways.

### **📌 Summary**
This project automates job searches on **Hirist** for **AI and Data Science roles**. The script:
1. **Fetches job listings dynamically** using web scraping.
2. Extracts **job details**, including **title, company, location, salary, and skills**.
3. **Presents data visually** using **graphs on skills, salaries, and job locations**.
4. **Uses GPT to generate insights** on job trends.
5. Allows users to **interactively search jobs** using a **Streamlit app**.

### **📌 Conclusion**
✅ This solution provides a **data-driven job search experience**.  
✅ **Real-time insights** help job seekers make informed decisions.  
✅ The **Streamlit interface** makes it user-friendly.  
✅ Future improvements can include **historical trend analysis** and **salary predictions**.  